In [39]:
import numpy as np
import pandas as pd
import math
import time
import re
import os
from tkinter import filedialog
from tkinter import *
from scipy.io import wavfile
from skimage import util
from scipy import signal
from scipy import stats

from sklearn.preprocessing import StandardScaler
#from sklearn.model_selection import KFold, train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score
from sklearn.decomposition import PCA

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

In [2]:
slices_path = 'C:/Users/Schindler/Documents/ProgrammingFun/Scripts/iPythonScripts/USV_python/600_annot_slices.csv'

In [3]:
data = pd.read_csv(slices_path)
annot_slices = pd.DataFrame(data = data)
print(annot_slices.shape)
annot_slices.head()

(552, 14136)


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,14125,14126,14127,14128,14129,14130,14131,14132,14133,14134
0,376560.0,0.057574,0.276893,0.528347,0.292352,0.967029,0.870557,0.288755,0.079022,1.101868,...,0.000009,0.000273,0.440910,0.040381,0.000693,0.084317,0.023230,0.015769,0.014675,0.077264
1,46305.0,0.011731,0.015517,0.001648,0.023396,0.003553,0.024199,0.257330,0.489177,0.373989,...,0.380229,0.000122,0.001736,0.007309,0.080183,0.083992,0.001830,0.020050,0.014924,0.059639
2,243270.0,0.008349,0.045459,0.472885,0.013214,0.057216,0.213565,0.059728,0.100463,0.006411,...,0.796693,0.005770,0.102972,0.008888,0.032086,0.006897,0.010392,0.002170,0.001961,0.031503
3,149692.5,0.011466,0.115345,0.000060,0.007044,0.003821,0.013232,0.260313,0.210111,0.069277,...,0.005605,0.030256,0.002974,0.038545,0.161589,0.043518,0.010613,0.001752,0.237444,0.009262
4,243157.5,0.024836,0.357126,0.115264,0.123585,0.186216,0.031609,0.192183,0.118203,3.902143,...,0.093640,0.081336,0.000161,0.051696,0.013878,0.016102,0.045677,0.040600,0.004259,0.042656


In [4]:
annot_slices.set_index('Unnamed: 0', inplace=True)
annot_slices.head()

,0,1,2,3,4,5,6,7,8,9,...,14125,14126,14127,14128,14129,14130,14131,14132,14133,14134
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
376560.0,0.057574,0.276893,0.528347,0.292352,0.967029,0.870557,0.288755,0.079022,1.101868,0.462441,...,0.000009,0.000273,0.440910,0.040381,0.000693,0.084317,0.023230,0.015769,0.014675,0.077264
46305.0,0.011731,0.015517,0.001648,0.023396,0.003553,0.024199,0.257330,0.489177,0.373989,0.130828,...,0.380229,0.000122,0.001736,0.007309,0.080183,0.083992,0.001830,0.020050,0.014924,0.059639
243270.0,0.008349,0.045459,0.472885,0.013214,0.057216,0.213565,0.059728,0.100463,0.006411,0.000584,...,0.796693,0.005770,0.102972,0.008888,0.032086,0.006897,0.010392,0.002170,0.001961,0.031503
149692.5,0.011466,0.115345,0.000060,0.007044,0.003821,0.013232,0.260313,0.210111,0.069277,0.003123,...,0.005605,0.030256,0.002974,0.038545,0.161589,0.043518,0.010613,0.001752,0.237444,0.009262
243157.5,0.024836,0.357126,0.115264,0.123585,0.186216,0.031609,0.192183,0.118203,3.902143,1.734283,...,0.093640,0.081336,0.000161,0.051696,0.013878,0.016102,0.045677,0.040600,0.004259,0.042656


In [56]:
feature_df = pd.DataFrame(index = annot_slices.index, columns = ['time_stamp', 'spec_flat', 'power_sum'], dtype=float)

for index, row in annot_slices.iterrows():

    feature_df.loc[index]['time_stamp'] = index
    #spectral flattness
    feature_df.loc[index]['spec_flat'] = stats.gmean(row) / row.mean()
    #power sum
    feature_df.loc[index]['power_sum'] = row.sum()

feature_df.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

,time_stamp,spec_flat,power_sum
Unnamed: 0,,,
376560.0,376560.0,0.155727,98748.960555
46305.0,46305.0,0.368722,32714.903093
243270.0,243270.0,0.232295,63014.446725
149692.5,149692.5,0.306856,43037.465935
243157.5,243157.5,0.062226,270193.532269


In [57]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 552 entries, 376560.0 to 239805.0
Data columns (total 3 columns):
time_stamp    546 non-null float64
spec_flat     546 non-null float64
power_sum     546 non-null float64
dtypes: float64(3)
memory usage: 37.2 KB


In [58]:
annot_path = "C:/Users/Schindler/Documents/Schindler_Lab/Data/Analysis/Excel files/USV/annot_info_df.xlsx"
data = pd.read_excel(annot_path)
annot_info = pd.DataFrame(data = data)
print(annot_info.shape)
annot_info.head()

(552, 6)


,Begin Time (s),Annotation,Animal,Group,Begin Time (s)_1000,time_stamp
0,376.574455,low slug,533,5,376574.45450,376560.0
1,46.306579,low slug,533,5,46306.57941,46305.0
2,243.272865,low slug,533,5,243272.86510,243270.0
3,149.708324,low slug,533,5,149708.32400,149692.5
4,243.176192,low slug,533,5,243176.19170,243157.5


In [59]:
annot_info.set_index('time_stamp', inplace=True)
annot_info.head()

,Begin Time (s),Annotation,Animal,Group,Begin Time (s)_1000
time_stamp,,,,,
376560.0,376.574455,low slug,533,5,376574.45450
46305.0,46.306579,low slug,533,5,46306.57941
243270.0,243.272865,low slug,533,5,243272.86510
149692.5,149.708324,low slug,533,5,149708.32400
243157.5,243.176192,low slug,533,5,243176.19170


In [60]:
features_with_info = pd.concat([annot_info, feature_df], axis=1)
print(features_with_info.shape)
features_with_info.head()

(552, 8)


,Begin Time (s),Annotation,Animal,Group,Begin Time (s)_1000,time_stamp,spec_flat,power_sum
376560.0,376.574455,low slug,533,5,376574.45450,376560.0,0.155727,98748.960555
46305.0,46.306579,low slug,533,5,46306.57941,46305.0,0.368722,32714.903093
243270.0,243.272865,low slug,533,5,243272.86510,243270.0,0.232295,63014.446725
149692.5,149.708324,low slug,533,5,149708.32400,149692.5,0.306856,43037.465935
243157.5,243.176192,low slug,533,5,243176.19170,243157.5,0.062226,270193.532269


In [61]:
features_with_info.groupby('Annotation')['power_sum'].describe()

,count,mean,std,min,25%,50%,75%,max
Annotation,,,,,,,,
bbc,16.0,2.040783e+05,1.933221e+05,65204.579293,118303.708439,157959.000681,179809.255671,8.720126e+05
high multi,5.0,2.495828e+05,2.766522e+05,36262.823360,82139.884439,85348.324129,353085.744318,6.910773e+05
high slug,9.0,2.744756e+05,2.536576e+05,33684.505775,35585.749435,241675.038673,423924.343339,6.687877e+05
low multi,16.0,3.965784e+05,3.698305e+05,45047.931416,175342.738599,243228.849389,552366.113060,1.366219e+06
low slug,60.0,1.042346e+05,1.514156e+05,29249.725489,45506.960588,59882.007608,111231.030926,1.161364e+06
noise,75.0,2.590662e+06,7.979640e+06,36285.484867,82726.628890,263591.988961,860112.283003,4.937721e+07
radar,365.0,1.398070e+05,9.293795e+04,30821.218661,86683.437453,123408.241084,161698.890938,1.145453e+06


In [62]:
features = features_with_info[['spec_flat', 'power_sum']]

In [63]:
# center and scale the data
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [64]:
k_range = range(2,20)
scores = []
for k in k_range:
    print(k)
    km_ss = KMeans(n_clusters=k, random_state=1)
    km_ss.fit(features)
    scores.append(silhouette_score(features, km_ss.labels_))
    print(scores)

# plot the results
plt.plot(k_range, scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')

2


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').